In [1]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off
!pip install groq --progress-bar off


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.2 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-aiplatform 1.56.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-bigquery-connection 1.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but y

In [2]:
import os
import textwrap
from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
os.environ["GROQ_API_KEY"] = ""

# def print_response(response):

def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))



# Document parsing

In [5]:
#Document parsing (cleaning the document)
instruction = """The provided document is the Constitution of Nepal 2072. This document encompasses all the legal frameworks, guidelines, and principles governing the country. It defines the structure of the state, the distribution of powers between different levels of government, and the fundamental rights and duties of citizens.

When answering questions based on this document, please follow these guidelines:
- Be precise and concise in your responses.
- Ensure that the information is accurate and directly relevant to the question.
- Highlight key articles, sections, or provisions where applicable.
- Provide context to your answers when necessary, explaining the implications or importance of specific provisions.
- Maintain a neutral and informative tone, avoiding any personal opinions or interpretations.

The goal is to provide clear and informative answers that help the user understand the specific aspects of the Constitution of Nepal 2072."""

parser = LlamaParse(
    api_key = '',
    result_type = 'markdown',
    parsing_instruction = instruction,
    max_timeout = 5000,

)

llama_parse_document = await parser.aload_data('/content/drive/My Drive/Constitution_Nepal.pdf')

Started parsing the file under job_id cac11eca-325d-4d71-86b4-75ec1194e38c


In [6]:
parsed_doc = llama_parse_document[0]
Markdown(parsed_doc.text[:1000])

# The Constitution of Nepal

|Date of Publication in Nepal Gazette|20 September 2015 (2072.6.3)|
|---|---|
|The Constitution of Nepal (First Amendment), 2016 (2072)|28 February 2016 (2072.11.16)|

Preamble:

We, the Sovereign People of Nepal, Internalizing the people's sovereign right and right to autonomy and self-rule, while maintaining freedom, sovereignty, territorial integrity, national unity, independence and dignity of Nepal, Recalling the glorious history of historic people's movements, armed conflict, dedication and sacrifice undertaken by the Nepalese people at times for the interest of the nation, democracy and progressive changes, and respecting for the martyrs and disappeared and victim citizens, Ending all forms of discrimination and oppression created by the feudalistic, autocratic, centralized, unitary system of governance, Protecting and promoting social and cultural solidarity, tolerance and harmony, and unity in diversity by recognizing the multi-ethnic, multi-lingua

In [7]:
from pathlib import Path

In [8]:
# save the file to
document_path = Path('/content/drive/MyDrive/LLM-Nepal-constitution/parsed_document.md')
with document_path.open("a") as f:
  f.write(parsed_doc.text)

# Vector embedding

In [9]:
# Handle markdown file
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
print(docs[0].page_content)

The Constitution of Nepal

Date of Publication in Nepal Gazette 20 September 2015 (2072.6.3) The Constitution of Nepal (First Amendment), 2016 (2072) 28 February 2016 (2072.11.16)

Preamble:

We, the Sovereign People of Nepal, Internalizing the people's sovereign right and right to autonomy and self-rule, while maintaining freedom, sovereignty, territorial integrity, national unity, independence and dignity of Nepal, Recalling the glorious history of historic people's movements, armed conflict, dedication and sacrifice undertaken by the Nepalese people at times for the interest of the nation, democracy and progressive changes, and respecting for the martyrs and disappeared and victim citizens, Ending all forms of discrimination and oppression created by the feudalistic, autocratic, centralized, unitary system of governance, Protecting and promoting social and cultural solidarity, tolerance and harmony, and unity in diversity by recognizing the multi-ethnic, multi-lingual, multi-religio

In [11]:
# model embedding
embeddings = FastEmbedEmbeddings(
    model_name = "BAAI/bge-base-en-v1.5")
qdrant = Qdrant.from_documents(docs,
                               embeddings,
                               path = '/content/drive/MyDrive/LLM-Nepal-constitution/qdrant_db',
                               collection_name = "document_embeddings")


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

In [12]:
%%time
query = "What fundamental rights are guaranteed to Nepali citizens?"
similar_docs = qdrant.similarity_search_with_score(query)

CPU times: user 297 ms, sys: 13.9 ms, total: 310 ms
Wall time: 312 ms


In [13]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("_" * 80)
    print()

text: Every person shall have the right to a fair trial by an independent, impartial and competent court or judicial body.

Any indigent party shall have the right to free legal aid in accordance with law.

Right of victim of crime:

(1) A victim of crime shall 

score: 0.7861023710448288
________________________________________________________________________________

text: Every person shall have the right to a fair trial by an independent, impartial and competent court or judicial body.

Any indigent party shall have the right to free legal aid in accordance with law.

Right of victim of crime:

(1) A victim of crime shall 

score: 0.7861023710448288
________________________________________________________________________________

text: Every person shall have the right to a fair trial by an independent, impartial and competent court or judicial body.

Any indigent party shall have the right to free legal aid in accordance with law.

Right of victim of crime:

(1) A victim of crime 

In [14]:
%%time
retriever = qdrant.as_retriever(search_kwargs = {"k": 5})
retrieved_docs = retriever.invoke(query)

CPU times: user 340 ms, sys: 48.7 ms, total: 388 ms
Wall time: 355 ms


In [15]:
for doc in retrieved_docs:
  print(f"id: {doc.metadata['_id']}\n")
  print(f"text: {doc.page_content[:256]}")
  print("_" * 80)
  print()

id: 48e4205b624b4cd699cb273bff8fc7ca

text: Every person shall have the right to a fair trial by an independent, impartial and competent court or judicial body.

Any indigent party shall have the right to free legal aid in accordance with law.

Right of victim of crime:

(1) A victim of crime shall 
________________________________________________________________________________

id: 0a10b67fe1844421a5be5d48bcda62e1

text: Every person shall have the right to a fair trial by an independent, impartial and competent court or judicial body.

Any indigent party shall have the right to free legal aid in accordance with law.

Right of victim of crime:

(1) A victim of crime shall 
________________________________________________________________________________

id: 5cdd3b59d44f4a83b0b4a54db31490b0

text: Every person shall have the right to a fair trial by an independent, impartial and competent court or judicial body.

Any indigent party shall have the right to free legal aid in accordance 

In [16]:
# path = '/content/drive/MyDrive/LLM-Nepal-constitution'

### Reranking

In [17]:
# Rerank and compress
compressor = FlashrankRerank(model = 'ms-marco-MiniLM-L-12-v2')
compression_retriever = ContextualCompressionRetriever(
    base_compressor = compressor,
    base_retriever = retriever)

ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:00<00:00, 50.5MiB/s]


In [18]:
%%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)

Running pairwise ranking..
CPU times: user 3.01 s, sys: 174 ms, total: 3.18 s
Wall time: 3.3 s


3

In [19]:
for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()

id: 48e4205b624b4cd699cb273bff8fc7ca

text: Every person shall have the right to a fair trial by an independent, impartial and competent court or judicial body.

Any indigent party shall have the right to free legal aid in accordance with law.

Right of victim of crime:

(1) A victim of crime shall 

score: 0.3514850437641144
--------------------------------------------------------------------------------

id: 0a10b67fe1844421a5be5d48bcda62e1

text: Every person shall have the right to a fair trial by an independent, impartial and competent court or judicial body.

Any indigent party shall have the right to free legal aid in accordance with law.

Right of victim of crime:

(1) A victim of crime shall 

score: 0.3514850437641144
--------------------------------------------------------------------------------

id: 5cdd3b59d44f4a83b0b4a54db31490b0

text: Every person shall have the right to a fair trial by an independent, impartial and competent court or judicial body.

Any indigent party

In [20]:
llm = ChatGroq(temperature = 0, model = 'llama3-70b-8192')

In [21]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [22]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose" : True},
)

In [23]:
%%time

response = qa.invoke("what are the fundamental rights guaranteed to Nepali citizens?")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Every person shall have the right to a fair trial by an independent, impartial and competent court or judicial body.

Any indigent party shall have the right to free legal aid in accordance with law.

Right of victim of crime:

(1) A victim of crime shall have pe right to get information about pe investigation and proceedings of a case in which he or she is pe victim.
(2) A victim of crime shall have pe right to justice including social rehabilitation and compensation in accordance wip law.

Right against torture:

No person who is arrested or detained shall be subjected to physical or mental torture or to cruel, inhuman or degrading treatment.

Right against preventive detention:

(1

In [24]:
print_response(response)

**Fundamental Rights Guaranteed to Nepali Citizens:**

Based on the provided information, the following fundamental rights are guaranteed to Nepali
citizens:

1. **Right to a Fair Trial**: Every person has the right to a fair trial by an independent,
impartial, and competent court or judicial body.
2. **Right to Free Legal Aid**: Any indigent party has the right to free legal aid in accordance
with law.
3. **Right of Victim of Crime**: A victim of crime has the right to:
        * Get information about the investigation and proceedings of a case.
        * Receive justice, including social rehabilitation and compensation, in accordance with law.
4. **Right against Torture**: No person who is arrested or detained shall be subjected to physical
or mental torture or to cruel, inhuman, or degrading treatment.
5. **Right against Preventive Detention**: No person shall be held under preventive detention unless
there is a sufficient ground of an immediate threat to the sovereignty, territoria

In [25]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose" : False},
)

In [26]:
%%time

response = qa.invoke("what are the fundamental rights guaranteed to Nepali citizens?")

Running pairwise ranking..
CPU times: user 2.24 s, sys: 171 ms, total: 2.41 s
Wall time: 3.48 s


In [27]:
print_response(response)

**Fundamental Rights Guaranteed to Nepali Citizens:**

Based on the provided information, the following fundamental rights are guaranteed to Nepali
citizens:

1. **Right to a Fair Trial**: Every person has the right to a fair trial by an independent,
impartial, and competent court or judicial body.
2. **Right to Free Legal Aid**: Any indigent party has the right to free legal aid in accordance
with law.
3. **Right of Victim of Crime**: A victim of crime has the right to:
        * Get information about the investigation and proceedings of a case.
        * Receive justice, including social rehabilitation and compensation, in accordance with law.
4. **Right against Torture**: No person who is arrested or detained shall be subjected to physical
or mental torture or to cruel, inhuman, or degrading treatment.
5. **Right against Preventive Detention**: No person shall be held under preventive detention unless
there is a sufficient ground of an immediate threat to the sovereignty, territoria

In [28]:
Markdown(response["result"])

**Fundamental Rights Guaranteed to Nepali Citizens:**

Based on the provided information, the following fundamental rights are guaranteed to Nepali citizens:

1. **Right to a Fair Trial**: Every person has the right to a fair trial by an independent, impartial, and competent court or judicial body.
2. **Right to Free Legal Aid**: Any indigent party has the right to free legal aid in accordance with law.
3. **Right of Victim of Crime**: A victim of crime has the right to:
	* Get information about the investigation and proceedings of a case.
	* Receive justice, including social rehabilitation and compensation, in accordance with law.
4. **Right against Torture**: No person who is arrested or detained shall be subjected to physical or mental torture or to cruel, inhuman, or degrading treatment.
5. **Right against Preventive Detention**: No person shall be held under preventive detention unless there is a sufficient ground of an immediate threat to the sovereignty, territorial integrity, or public peace and order of Nepal.
6. **Right against Untouchability and Discrimination**: No person shall be subjected to any form of untouchability or discrimination in any private and public places on grounds of origin, caste, tribe, community, profession, occupation, or physical condition.

These fundamental rights aim to protect the dignity, freedom, and well-being of Nepali citizens, ensuring they are treated fairly and without discrimination.

In [29]:
%%time
response = qa.invoke("What are the powers and responsibilities of federal, provincial, and local governments?")
Markdown(response["result"])



Running pairwise ranking..
CPU times: user 2 s, sys: 118 ms, total: 2.12 s
Wall time: 3.09 s


Based on the provided information, I don't know the specific powers and responsibilities of federal, provincial, and local governments. The text only discusses the executive power of the Local Level, the Village Executive, and the Municipal Executive, as well as the provisions relating to the Chairperson and Vice-Chairperson of the Village Executive.

However, I can provide some general information about the executive power of the Local Level:

* The executive power of the Local Level is vested in the Village Executive or the Municipal Executive.
* The local executive power is as mentioned in Schedule-8 and Schedule-9.
* The Village Executive and the Municipal Executive are responsible for issuing general directives, controlling, and regulating the governance of the Village Body and the Municipality.
* The executive functions of the Village Body and the Municipality are performed in the name of the Village Executive and the Municipal Executive.

If you have any further questions or need clarification on any of these points, feel free to ask!

In [30]:
%%time
response = qa.invoke("Who is Binit")
Markdown(response["result"])


Running pairwise ranking..
CPU times: user 3.51 s, sys: 89.3 ms, total: 3.6 s
Wall time: 5.95 s


I don't know who Binit is. The provided information does not mention a person named Binit. It appears to be a set of rules and regulations related to the State Government, Chief Minister, and Ministers, but it does not contain any information about a person named Binit.

In [31]:
%%time
response = qa.invoke("how has the constitution of nepal divided the revenue under federalism?")
Markdown(response["result"])


Running pairwise ranking..
CPU times: user 1.74 s, sys: 114 ms, total: 1.85 s
Wall time: 3.18 s


**Answer:** The Constitution of Nepal has divided the revenue under federalism in a way that:

* The Federation, State, and Local level may impose taxes on matters falling within their respective jurisdiction and collect revenue from these sources.
* The Government of Nepal shall make provisions for the equitable distribution of the collected revenue to the Federation, State, and Local level.
* The amount of fiscal transfer receivable by the State and Local level shall be as recommended by the National Natural Resources and Fiscal Commission.
* The Government of Nepal shall distribute fiscal equalization grants to the State and Local level based on the need of expenditure and revenue capacity.
* Each State shall distribute fiscal equalization grants to its subordinate Local level based on the need of expenditure and revenue capacity.
* Provisions relating to distribution of conditional grants, complementary grants, or special grants for other purposes shall be as provided for in the Federal law.

**Additional helpful information:**

* The distribution of revenues between the Federal, State, and Local level shall be made in a balanced and transparent manner.
* The Government of Nepal has the authority to determine provisions relating to the imposition of taxes and collection of revenue on matters that fall within the Concurrent List and on matters that are not included in the List of any level.

In [32]:
%%time
response = qa.invoke("list out the directive principles of state under constitution of nepal?")
Markdown(response["result"])


Running pairwise ranking..
CPU times: user 2.06 s, sys: 121 ms, total: 2.18 s
Wall time: 11.6 s


Based on the provided information, the directive principles of the State under the Constitution of Nepal are:

**Political Objectives:**

1. Establish a public welfare system of governance
2. Establish a just system in all aspects of national life through the rule of law, values, and norms of fundamental rights and human rights
3. Ensure gender equality, proportional inclusion, participation, and social justice
4. Protect the life, property, equality, and liberties of the people
5. Consolidate a federal democratic republican system of governance
6. Maintain relations between Federal Units on the basis of cooperative federalism
7. Incorporate the principle of proportional participation in the system of governance on the basis of local autonomy and decentralization

**Social and Cultural Objectives:**

1. Build a civilized and egalitarian society
2. Eliminate all forms of discrimination, exploitation, and injustice on various grounds
3. Develop social, cultural values founded on national pride, democracy, pro-people, respect of labor, entrepreneurship, discipline, dignity, and harmony
4. Consolidate national unity by maintaining social cohesion, solidarity, and harmony while recognizing cultural diversity

Note: These objectives are guiding principles for the governance of the State, and the State shall mobilize means and resources to implement these principles, policies, and obligations.

In [34]:
%%time
response = qa.invoke("What are punishment if someone performs rape?")
Markdown(response["result"])

Running pairwise ranking..
CPU times: user 1.01 s, sys: 47.7 ms, total: 1.06 s
Wall time: 1.49 s


**I don't know**

The provided context does not mention rape or its punishment. It lists various acts that are prohibited, such as treason, incitement to hatred, and undermining harmonious relations between different groups, but it does not mention rape or any related punishment.

In [35]:
%%time
response = qa.invoke("Does Article 29, which guarantees the right to religious freedom, also protect the right to proselytize? Explain your reasoning based on the text and relevant legal precedents.")
Markdown(response["result"])

Running pairwise ranking..
CPU times: user 2.56 s, sys: 105 ms, total: 2.66 s
Wall time: 3.65 s


**Answer:** No, Article 29, which guarantees the right to religious freedom, does not protect the right to proselytize.

**Reasoning:** Clause (3) of Article 29 explicitly states that "No person shall, in the exercise of the right conferred by this Article, do, or cause to be done, any act which may be contrary to public health, decency and morality or breach public peace, or convert another person from one religion to another..." This implies that the right to religious freedom does not extend to proselytizing or converting others from one religion to another.

**Additional Information:** This provision is likely intended to maintain social harmony and prevent religious conflicts. It is also consistent with the principles of religious tolerance and coexistence.

In [33]:
from langchain.agents import initialize_agent, Tool

def constitution_lookup(query: str) -> str:
  results = retriever.get_relevant_documents(query)
  return "\n".join([doc.page_content for doc in results])

constitution_lookup_tool = Tool(
    name = "Constitution Lookup",
    func = constitution_lookup,
    description = "Retrieves relevant sections from the Constitution."
)

# Initalize the agent
agent = initialize_agent(
    tools = [constitution_lookup_tool],
    llm = llm,
    agent = "zero-shot-react-description",
    verbose = True
)

In [ ]:
while True:
  user_query = input()

In [38]:
# Define your tool
def constitution_lookup(query: str) -> str:
    results = retriever.get_relevant_documents(query)
    return "\n".join([doc.page_content for doc in results])

constitution_lookup_tool = Tool(
    name="Constitution Lookup",
    func=constitution_lookup,
    description="Retrieves relevant sections from the Constitution."
)

# Initialize the agent
agent = initialize_agent(
    tools=[constitution_lookup_tool],
    llm=llm,  # Your ChatGroq model
    agent="zero-shot-react-description",
    verbose=True
)

# Interaction Loop
while True:
    user_query = input("Ask a question about the Constitution: ")
    response = agent.run(user_query)
    print(response)

Ask a question about the Constitution: WHAT ARE THE FUNDAMENTAL RIGHT


> Entering new AgentExecutor chain...
Thought: I need to understand what the fundamental rights are in the context of the US Constitution.

Action: Constitution Lookup
Action Input: Search for sections related to fundamental rights

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



Observation: Right to privacy

The privacy of any person, his or her residence, property, document, data, correspondence and matters relating to his or her character shall, except in accordance with law, be inviolable.

Right against exploitation

Every person shall have the right against exploitation.

No person shall be exploited in any manner on the grounds of religion, custom, tradition, usage, practice or on any other grounds.

No one shall be subjected to trafficking nor shall one be held in slavery or servitude.

No one shall be forced to work against his or her will.

Provided that nothing shall be deemed to prevent the making of law empowering the State to require citizens to perform compulsory service for public purposes.

Act contrary to clauses (3) and (4) shall be punishable by law and the victim shall have the right to obtain compensation from the perpetrator in accordance with law.

Right to clean environment

Every citizen shall have the right to live in a clean and he

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01j15cnh3devrvfjas213t26d0` on tokens per minute (TPM): Limit 6000, Used 4722, Requested 7630. Please try again in 1m3.512s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [39]:
%%time
response = qa.invoke("Does Article 29, which guarantees the right to religious freedom, also protect the right to proselytize? Explain your reasoning based on the text and relevant legal precedents.")
Markdown(response["result"])

Running pairwise ranking..
CPU times: user 2.64 s, sys: 112 ms, total: 2.75 s
Wall time: 3.64 s


**Answer:** No, Article 29, which guarantees the right to religious freedom, does not protect the right to proselytize.

**Reasoning:** Clause (3) of Article 29 explicitly states that "No person shall, in the exercise of the right conferred by this Article, do, or cause to be done, any act which may be contrary to public health, decency and morality or breach public peace, or convert another person from one religion to another..." This implies that the right to religious freedom does not extend to proselytizing or converting others from one religion to another.

**Additional Information:** This provision is likely intended to maintain social harmony and prevent religious conflicts. It is also consistent with the principles of religious tolerance and coexistence.